In [8]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy

from datetime import datetime
import pickle

import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.utils import Progbar

In [9]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
mode = 'test'
name = 'sarsa'
level = 1
if level == 2:
    name += name + 'lv2'

base_path='/content/drive/MyDrive/trade_rl/'

In [22]:
df=pd.read_csv(base_path+"BTC_N=60_rosoku.csv",index_col=1)
df= df.iloc[:,1:]
df

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
1546300800,3693.85,3703.25,3683.45,3694.36,94.466858,3.488331e+05,3692.850555
1546304400,3692.89,3699.35,3676.13,3681.18,101.456816,3.743185e+05,3688.716991
1546308000,3681.18,3683.55,3665.00,3680.84,108.184205,3.975652e+05,3678.169693
1546311600,3680.84,3686.73,3678.12,3682.12,181.350960,6.676729e+05,3681.311417
1546315200,3681.10,3702.04,3679.63,3679.80,189.394057,6.979663e+05,3684.028641
...,...,...,...,...,...,...,...
1564394400,9460.24,9507.00,9433.12,9484.26,221.363522,2.099191e+06,9479.491527
1564398000,9475.10,9498.89,9418.55,9451.67,147.829088,1.398556e+06,9467.027268
1564401600,9452.30,9514.96,9433.95,9503.19,316.997002,3.003061e+06,9467.400217


In [23]:
class Environment:
    def __init__(self,df,wallet=[10**4,0]):
        #experiment_setting
        self.init_wallet=wallet[:]
        self.wallet=self.init_wallet[:]
        self.action_space=[0,1,2]
        self.window_size=15

        self.total_steps=len(df)-self.window_size-1

        

        #situation
        self.reset()
        
    def reset(self):
        self.now_step=0

        self.end_step= self.total_steps
        self.wallet=self.init_wallet[:]
        #price
        self.now_price=0

        #trade
        self.trade_time=0

        #log
        self.data_time=0
        return self._get_now_state()

    def step(self,action):
        self.now_step+=1

        done=(self.now_step+self.window_size+1==self.end_step)
        now_price=df["Open"].values[self.now_step+self.window_size+1]
        #print(self.now_step+self.window_size+1,len(df))

        self._trade(action,now_price)
        reward=self.wallet[0]+self.wallet[1]*now_price-sum(self.init_wallet)
        info={"cur_revenue":reward,"trade_time":self.trade_time}

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        #state = np.empty(0)
        #過去の値動きをflattenにして入力したい
        
        #state[0] = self.hold_a_position
        start=time.time()
        price_data=np.array(
            df[self.now_step:self.now_step+self.window_size]).flatten()
        spend=time.time()-start
        self.data_time+=spend

        return price_data

    def _trade(self,action,price):
        pena=0.01
        if action==0:#Hold
            pass
        elif action==1:#buy
            if self.wallet[0]!=0:
                self.wallet[1]+=self.wallet[0]/price*(1-pena)
                self.wallet[0]=0
                self.trade_time+=1
        elif action==2:#sell
            if self.wallet[1]!=0:
                self.wallet[0]+=self.wallet[1]*price*(1-pena)
                self.wallet[1]=0
                self.trade_time+=1
        else:
            print("actionがおかしいぞ")

In [24]:
class Brain:
    def __init__(self):

        self.gamma = 0.9
        optimizer = RMSprop()

        model = Sequential()
        model.add(Dense(3, input_shape=(105,)))
        model.add(ReLU()) 
        model.add(Dense(3))
        model.add(ReLU())
        model.add(Dense(3))
        model.add(ReLU())
        model.add(Dense(3))
        model.compile(loss="mse", optimizer=optimizer)

        print((model.summary()))
        self.model = model

    def train(self, state, action, reward, next_state, done):
        q = self.model.predict(state,verbose=0)
        next_q = self.model.predict(next_state,verbose=0)
        target = np.copy(q)
        if done:
            target[:, action] = reward
        else:
            target[:, action] = reward + self.gamma*np.max(next_q, axis=1)
        self.model.train_on_batch(state, target)

    def predict(self, state):
        return self.model.predict(state,verbose=0)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [25]:
class Agent(Brain):
    def __init__(self):

        super().__init__()
        self.epsilon = 0.1
        self.epsilon_min = 0.001
        self.r = 0.8
        return

    def act(self, state, act, reward, info, next_state, done, mode = 'train'):
        next_act = self._next_act(state)
        if mode == 'train':
            self.train(state, act, reward, next_state, next_act, done)

        return next_act

    def _next_act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(3)
        act_values = self.predict(state)

            
        return np.argmax(act_values[0])

In [26]:
class Main:
    def __init__(self, env, agent, mdl_dir, name, episodes_times = 1000, mode = 'test'):
        self.env = env
        self.agent = agent
        self.mdl_dir = mdl_dir
        self.scaler = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.mode = mode
        self.name = name

        self.df_rec = pd.DataFrame(index=[], columns=['FixedProfit',"TradeTimes"])
        if self.mode == 'test':
            self.agent.epsilon = 0.01

    def play_game(self):

        for episode in range(self.episodes_times):
            
            if (episode % 10 == 0):
                metrics_names = ['FixedProfit']
                if (int(str(self.episodes_times)[:-1])*10 == episode):
                    pb_i = Progbar(int(str(self.episodes_times)[-1]), stateful_metrics=metrics_names)
                else:
                    pb_i = Progbar(10, stateful_metrics=metrics_names)
                p_mean = np.array([])
                trade_time = np.array([])
            
            state = self.env.reset()
            state = self.scaler.transform([state])
            done = False
            reward = 0.0
            info = None
            next_state = copy.copy(state)
            next_act = 0
            act = 0
            
            
            while not done:
                start=time.time()
                action = self.agent.act(state, act, reward, info, next_state, done, mode)
                state = next_state
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])
                #reward = self._reward_clipping(reward)
                act = copy.copy(action)
                state = next_state

                if self.mode == 'train':
                    self.agent.train(state, action, reward, next_state, done)

            if self.agent.epsilon > self.agent.epsilon_min:
                self.agent.epsilon *= self.agent.r

            print("epsilon:%f"%self.agent.epsilon)

            record = pd.Series([info['cur_revenue'],info["trade_time"]], index=self.df_rec.columns)
            self.df_rec = self.df_rec.append(record, ignore_index=True)
            p_mean,trade_time = np.append(p_mean,info['cur_revenue']),np.append(trade_time,info['trade_time'])
            values=[('FixedProfit',int(np.mean(p_mean))), ('TradeTimes',int(np.mean(trade_time)))]
            pb_i.add(1, values=values)
            
            if self.mode == 'train':
                self._save()
            self._save_csv()
        

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _reward_clipping(self, val):
        result = 1 if val > 0 else 0 if val == 0 else -1
        return result

    def _load(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'rb') as f:
            self.scaler = pickle.load(f)
        self.agent.load('{}/{}.h5'.format(self.mdl_dir, self.name))

    def _save(self):
        self.agent.save('{}/{}.h5'.format(self.mdl_dir, self.name))
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'wb') as f:
            pickle.dump(self.scaler, f)

    def _save_csv(self):
        now=time.time()
        self.df_rec.to_csv(base_path+"log_sarsa.csv")

In [ ]:
initial_money=1000000
episodes_times = 300
commission = 0 if level == 1 else 0.002

agent = Agent()
env = Environment(df)
main = Main(env, agent, ".", name, episodes_times, mode="train")
main.play_game()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 3)                 318       
                                                                 
 re_lu_3 (ReLU)              (None, 3)                 0         
                                                                 
 dense_5 (Dense)             (None, 3)                 12        
                                                                 
 re_lu_4 (ReLU)              (None, 3)                 0         
                                                                 
 dense_6 (Dense)             (None, 3)                 12        
                                                                 
 re_lu_5 (ReLU)              (None, 3)                 0         
                                                                 
 dense_7 (Dense)             (None, 3)                